In [ ]:
import subprocess
smi=subprocess.run(["nvidia-smi", "-L"], stdout=subprocess.PIPE)
if smi.returncode != 0:
  assert False, "No GPU available!"
print(smi.stdout.decode())

GPU 0: Tesla T4 (UUID: GPU-0b740607-f6b1-1f65-9817-d5606c3c623b)



In [ ]:
# SSH configuration
NGROK_TOKEN = 'your_ngrok_token'
USER_PASSWORD = 'your_user_password'

# Constant
KATAGO_COLAB_REPOSITORY_URL = 'https://github.com/matobataketoshi/dlshogi-colab.git'
KATAGO_RESOURCE_FILE = '/content/dlshogi-colab/colab-resource/external-resource.json'
SSH_INFO_FILE_NAME = 'colab-katago-ssh.json'
SSH_INFO_DIR = '/content/drive/MyDrive'

# Install useful stuff
!echo "必要なライブラリをインストールします。"
!apt-get update 1>/dev/null
!apt-get install --yes ssh screen nano htop ranger git libzip4 1>/dev/null
!pip install -U -q PyDrive 1>/dev/null

import subprocess
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import requests
import json
import os
from re import sub

def get_katago_resource(katago_resource_file):
    with open(katago_resource_file, 'r') as f:
        resource = json.load(f)
    return resource

def save_ssh_info(dir, file_name):
    file_path = dir + '/' + file_name
    ssh_option = create_ssh_option()
    file_id = get_file_id_in_drive(file_path)
    if file_id is None:
        file_id = upload_ssh_option_with_pydrive(file_name, ssh_option)
    else:
        ssh_option['fileId'] = file_id
        write_ssh_option(file_path, ssh_option)
    print('================================================================')
    print('SSH_INFO_GOOGLE_DRIVE_FILE_ID: {}'.format(file_id))
    print('================================================================')
    return file_id

def create_ssh_option():
    ssh_option = None
    try:
        r = requests.get('http://localhost:4040/api/tunnels')
        raw_ssh = r.json()['tunnels'][0]['public_url']
        ssh_args = (sub("tcp://", "", raw_ssh)).split(':')
        ssh_option = {
            'host': ssh_args[0],
            'port': int(ssh_args[1]),
            'user': 'root'
        }
    except Exception as e:
        print('Failed to create SSH options. Make sure that the ngrok token is set correctly.')
        raise e
    return ssh_option

def get_file_id_in_drive(file_path):
    file_id = None
    if (os.path.exists(file_path)):
        try:
            with open(file_path, 'r') as f:
                ssh_option = json.load(f)
                file_id = ssh_option.get('fileId')
        except Exception as e:
            file_id = None
    return file_id

def write_ssh_option(file_path, ssh_option):
    print('Write SSH option: {}'.format(file_path))
    with open(file_path, 'w') as f:
        f.write(json.dumps(ssh_option))
 
def upload_ssh_option_with_pydrive(file_name, ssh_option):
    print('Authenticate to Google Drive')
    drive = auth_google_drive()
    file_metadata = {
        'title': file_name,
        'mimeType': 'application/json'
    }
    file_list = drive.ListFile({'q': 'title="' + file_name + '" and trashed=False'}).GetList()
    if (len(file_list) > 0):
        file_metadata['id'] = file_list[0]['id']
    ssh_option_file = drive.CreateFile(file_metadata)
    ssh_option_file.Upload()
    file_id = ssh_option_file.get('id')
    ssh_option['fileId'] = file_id
    ssh_option_file.SetContentString(json.dumps(ssh_option))
    ssh_option_file.Upload()
    ssh_option_file.InsertPermission({'type': 'anyone', 'value': 'anyone', 'role': 'reader'})
    return file_id

def auth_google_drive():
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    return drive

%cd /content
# Clone katago-colab
!echo "Gitから必要なファイルを受け取ります。"
!rm -rf katago-colab
!git clone $KATAGO_COLAB_REPOSITORY_URL 1>/dev/null

# Get URLs of external resources
katago_resource = get_katago_resource(KATAGO_RESOURCE_FILE)
ngrok_url = katago_resource['ngrok']

# Download ngrok
!wget -q $ngrok_url -O ngrok
!chmod +x /content/ngrok
 
# SSH setting
!echo "root:$USER_PASSWORD" | chpasswd
!echo "PasswordAuthentication yes" > /etc/ssh/sshd_config
!echo "PermitUserEnvironment yes" >> /etc/ssh/sshd_config
!echo "PermitRootLogin yes" >> /etc/ssh/sshd_config

!mkdir -p /root/.ssh
!service ssh restart > /dev/null
 
# Run ngrok
!echo "ngrokを実行します。"
get_ipython().system_raw('./ngrok authtoken $NGROK_TOKEN && ./ngrok tcp 22 &')
!sleep 5

# Save SSH info to Google Drive
!echo "GoogleドライブにSSH接続情報を保存します。"
SSH_INFO_GOOGLE_DRIVE_FILE_ID = save_ssh_info(SSH_INFO_DIR, SSH_INFO_FILE_NAME)

%cd /content
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/libcudnn8_8.2.1.32-1+cuda11.3_amd64.deb
!dpkg -i libcudnn8_8.2.1.32-1+cuda11.3_amd64.deb

!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/libnvinfer8_8.2.2-1+cuda11.4_amd64.deb
!dpkg -i libnvinfer8_8.2.2-1+cuda11.4_amd64.deb
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/libnvonnxparsers8_8.2.2-1+cuda11.4_amd64.deb
!dpkg -i libnvonnxparsers8_8.2.2-1+cuda11.4_amd64.deb
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/libnvparsers8_8.2.2-1+cuda11.4_amd64.deb
!dpkg -i libnvparsers8_8.2.2-1+cuda11.4_amd64.deb
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/libnvinfer-plugin8_8.2.2-1+cuda11.4_amd64.deb
!dpkg -i libnvinfer-plugin8_8.2.2-1+cuda11.4_amd64.deb

# Download KataGo binary
!echo "dlshogi実行ファイルを受け取ります。"
!wget -q https://github.com/matobataketoshi/dlshogi-colab/releases/download/dlshogi/dlshogi -O dlshogi
!chmod +x /content/dlshogi
!/content/dlshogi version

#!wget https://github.com/TadaoYamaoka/DeepLearningShogi/releases/download/dr2_exhi/model-dr2_exhi.zip
#!unzip -P dlshogi-dr2_exhi model-dr2_exhi.zip

# Copy enviroment variable for SSH
with open('/root/.ssh/environment','w') as f:
    for key, value in os.environ.items():
        print('{}={}'.format(key, value), file=f)

!wget https://github.com/TadaoYamaoka/DeepLearningShogi/releases/download/dr2_exhi/model-dr2_exhi.zip
!unzip -P dlshogi-dr2_exhi model-dr2_exhi.zip

!./dlshogi

!echo "接続の準備が整いました! lizzieを接続できます..."
!echo "lizzie engine 駆動命令 : .\colab-katago-gd.exe $SSH_INFO_GOOGLE_DRIVE_FILE_ID $USER_PASSWORD"

In [ ]:
%cd /content/drive/MyDrive
!cp -r /content/DeepLearningShogi DeepLearningShogi

/content/drive/MyDrive


In [ ]:
#setoption name DNN_Model value /content/model-dr2_exhi.onnx
#setoption name UCT_Threads value 3
#isready

In [ ]:
#if readyok

#position startpos
#go byoyomi 5000

In [ ]:
!sudo python3 -mpip install -U cshogi